In [ ]:
import requests
import pandas as pd

ano = '2024'
ente = '33'

############################## IMPORTANDO os ANEXOS do RREO (6o Bimentre)
## Anexo 1
link_rreo_1 = f'https://apidatalake.tesouro.gov.br/ords/siconfi/tt/rreo?an_exercicio={ano}&nr_periodo=6&co_tipo_demonstrativo=RREO&no_anexo=RREO-Anexo%2001&id_ente={ente}'
rreo_1 = requests.get(link_rreo_1)
df_rreo_1 = rreo_1.json()
df_rreo_1 = pd.DataFrame(df_rreo_1["items"])

## Anexo 2 >> Demonstrativo da Execução da Despesa por Função/Subfunção (Anexo 02)
link_rreo_2 = f'https://apidatalake.tesouro.gov.br/ords/siconfi/tt/rreo?an_exercicio={ano}&nr_periodo=6&co_tipo_demonstrativo=RREO&no_anexo=RREO-Anexo%2002&id_ente={ente}'
rreo_2 = requests.get(link_rreo_2)
df_rreo_2 = rreo_2.json()
df_rreo_2 = pd.DataFrame(df_rreo_2["items"])


st.write("## 📊 Dashboards - Demonstrativos Fiscais")

st.markdown("""
<div class="card">
  <p><b>Painéis visuais e gerenciais</b> com base nos demonstrativos enviados ao SICONFI. Permite análises detalhadas
     da execução orçamentária e fiscal do ente, com visualizações interativas e KPIs.</p>

  <div class="page-sep"></div>

  <p><b>Funcionalidades Atuais:</b></p>
  <ul>
    <li><b>RREO 1 (Balanço Orçamentário):</b> Execução de receitas e despesas com comparativos</li>
    <li><b>RREO 2 (Despesas por Função):</b> Análise das despesas por funções governamentais</li>
  </ul>

  <div class="page-sep"></div>

  <p><b>Em Desenvolvimento:</b> Expansão para incluir indicadores de RCL, Dívida Consolidada, Gastos de Pessoal e
     Previdenciários, seguindo o modelo dos relatórios "RREO em Foco" e "RGF em Foco" da STN.</p>
</div>
""", unsafe_allow_html=True)

st.divider()



# ------------------------------------------------------------
    tab_rreo01, tab_rreo02, tab_series, tab_tabelas = st.tabs([
        "Balanço Orçamentário (RREO 1)",
        "Execução por Função (RREO 2)",
        "📈 Séries",
        "🧾 Tabelas",
    ])

    # ============================================================
    # TAB RREO 1
    # ============================================================
    with tab_rreo01:
        st.write("## RECEITA")
        st.subheader("Receita — Previsão × Arrecadação")

        def _fmt_brl(v: float) -> str:
            return "R$ " + f"{v:,.0f}".replace(",", ".")

        # KPIs diretamente da linha TOTAL oficial
        df_r1 = bases.get("rreo", {}).get("1", pd.DataFrame()).copy()

        def _pick_total_code(df: pd.DataFrame) -> str | None:
            if df.empty or "cod_conta" not in df:
                return None
            un = set(df["cod_conta"].astype(str).unique())
            for cand in ["Receitas", "TotalReceitas", "ReceitasExcetoIntraOrcamentarias"]:
                if cand in un:
                    return cand
            for u in un:
                if "RECEITA" in u.upper():
                    return u
            return None

        def _pick_col(df: pd.DataFrame, want: str) -> list[str]:
            # want: "A" ou "C"
            patsA = ["ATUALIZAD", "(A)"]
            patsC = ["BIMESTRE", "(C)"]
            Pats = patsA if want == "A" else patsC
            cols = df["coluna"].astype(str)
            chosen = cols[cols.str.upper().apply(lambda t: all(p in t for p in Pats))]
            if not chosen.empty:
                return list(chosen.unique())
            if want == "A":
                return list(cols[cols.str.contains(r"\(a\)", case=False, regex=True)].unique())
            else:
                return list(cols[cols.str.contains(r"\(c\)", case=False, regex=True)].unique())

        cod_total_receita = _pick_total_code(df_r1)
        if cod_total_receita:
            labels_A = _pick_col(df_r1, "A")
            labels_C = _pick_col(df_r1, "C")
            a = float(df_r1.query('cod_conta == @cod_total_receita and coluna in @labels_A')["valor"].sum())
            c = float(df_r1.query('cod_conta == @cod_total_receita and coluna in @labels_C')["valor"].sum())
        else:
            a = c = 0.0

        saldo = max(a - c, 0.0)
        perc = (c / a * 100) if a else 0.0

        # KPIs
        colA, colB, colC = st.columns(3)
        colA.metric("Previsão Atualizada (a)", _fmt_brl(a))
        colB.metric("Arrecadado até o bimestre (c)", _fmt_brl(c))
        colC.metric("Saldo (a-c)", _fmt_brl(saldo))

        # Cores
        COR_PREV = "#A925EB"
        COR_ARREC = "#3030CE"
        COR_SALDO = "#16A382"
        CINZA_BG = "#CBD5E1"

        # Anel
        col_g1, col_g2 = st.columns([1, 1])
        with col_g1:
            ring = go.Figure(go.Pie(
                values=[c, max(a - c, 0)],
                labels=["Arrecadado", "A arrecadar"],
                hole=0.72, sort=False, direction="clockwise",
                marker=dict(colors=[COR_ARREC, CINZA_BG], line=dict(width=0)),
                textinfo="none",
                hovertemplate="%{label}: <b>R$ %{value:,.0f}</b><extra></extra>",
                showlegend=False,
            ))
            ring.update_traces(rotation=90)
            ring.update_layout(
                template="simple_white",
                height=340, margin=dict(l=10, r=10, t=10, b=10),
                annotations=[
                    dict(text=f"{perc:,.1f}%", x=0.5, y=0.54, showarrow=False,
                         font=dict(size=28, color="#111827", family="Inter, system-ui")),
                    dict(text="do previsto", x=0.5, y=0.35, showarrow=False,
                         font=dict(size=12, color="#6B7280")),
                ],
            )
            st.plotly_chart(ring, use_container_width=True)

        # Barras
        with col_g2:
            labels = ["Previsão Atualizada (a)", "(-) Arrecadado até o bimestre (c)", "Saldo (a-c)"]
            vals = [a, c, saldo]
            cores = [COR_PREV, COR_ARREC, COR_SALDO]
            fig = go.Figure(go.Bar(
                x=labels, y=vals, marker_color=cores,
                text=[_fmt_brl(v) for v in vals],
                textposition="outside",
                cliponaxis=False
            ))
            ymax = max(vals) * 1.20 if vals else 0
            tickvals, ticktext, top_round, _ = auto_ticks(ymax, max_ticks=8)
            fig.update_layout(
                template="simple_white",
                height=380,
                margin=dict(l=20, r=20, t=56, b=32),
                showlegend=False,
                font=dict(family="Inter, system-ui"),
                plot_bgcolor="rgba(0, 0, 0, 0)"
            )
            fig.update_yaxes(
                title="", range=[0, top_round],
                tickmode="array", tickvals=tickvals, ticktext=ticktext,
                ticks="outside", tickfont=dict(size=12),
                zeroline=True, zerolinecolor="rgba(0,0,0,0.1)",
                showgrid=True
            )
            st.plotly_chart(fig, use_container_width=True)

        # --- Top 10 Receitas (detalhado) ---
        if not rec.empty and "arrec_acum" in rec.columns:
            st.subheader("Maiores Receitas Arrecadadas (acum.)")

            df = rec.copy()
            df["conta"] = df["conta"].astype("string").str.strip()
            df["cod_conta"] = df["cod_conta"].astype("string").str.strip()
            df["arrec_acum_num"] = pd.to_numeric(df["arrec_acum"], errors="coerce")

            m_val = df["arrec_acum_num"].notna() & (df["arrec_acum_num"] > 0)
            eh_agregador_codigo = df["cod_conta"].str.contains(
                r"^(Total|Subtotal|Saldo|Resultado)", case=False, na=False
            )
            tem_minuscula = df["conta"].str.contains(r"[a-záéíóúâêôãõç]", regex=True, na=False)

            folhas = df[m_val & tem_minuscula & ~eh_agregador_codigo].copy()
            if folhas.empty:
                st.info("Não há contas de receita detalhadas para exibir.")
            else:
                base = folhas.groupby("conta", as_index=False)["arrec_acum_num"].sum()

                top = (base
                       .nlargest(10, "arrec_acum_num")
                       .sort_values("arrec_acum_num", ascending=False))

                fig_topR = px.bar(
                    top, x="arrec_acum_num", y="conta", orientation="h",
                    labels={"arrec_acum_num": "Arrecadado (R$)", "conta": ""}
                )

                fig_topR.update_yaxes(
                    categoryorder="array",
                    categoryarray=top["conta"].tolist(),
                    autorange="reversed"
                )

                ymax = float(top["arrec_acum_num"].max()) * 1.10
                candidatos = [1, 2, 5, 10, 20, 50, 100, 200]
                passo = next((p for p in candidatos if ymax / (p * 1e6) <= 8), 200) * 1e6
                unidade = "Mi"
                if ymax >= 1e9:
                    passo = next((p for p in candidatos if ymax / (p * 1e9) <= 8), 200) * 1e9
                    unidade = "Bi"

                top_round = math.ceil(ymax / passo) * passo
                tickvals = [i * passo for i in range(int(top_round / passo) + 1)]
                ticktext = ["0" if v == 0 else f"{int(v/(1e9 if unidade=='Bi' else 1e6))} {unidade}" for v in tickvals]

                fig_topR.update_layout(
                    template="simple_white",
                    height=520,
                    margin=dict(l=12, r=16, t=10, b=10)
                )
                fig_topR.update_xaxes(
                    title="Arrecadado (R$)",
                    range=[0, top_round],
                    tickvals=tickvals, ticktext=ticktext,
                    zeroline=True, zerolinecolor="rgba(0,0,0,0.1)"
                )

                st.plotly_chart(fig_topR, use_container_width=True)

        st.divider()

        # --------- DESPESA RREO 1 (totais da linha oficial) ---------
        st.write("## DESPESA")
        st.subheader("Despesa — Empenhado → Liquidado → Pago")

        DOT_ATUAL = "DOTAÇÃO ATUALIZADA (e)"
        EMP_ACUM = "DESPESAS EMPENHADAS ATÉ O BIMESTRE (f)"
        LIQ_ACUM = "DESPESAS LIQUIDADAS ATÉ O BIMESTRE (h)"
        PAG_ACUM = "DESPESAS PAGAS ATÉ O BIMESTRE (j)"

        def _tot_desp(rotulo: str) -> float:
            return float(df_r1.query('cod_conta == "TotalDespesas" and coluna == @rotulo')["valor"].sum())

        dot = _tot_desp(DOT_ATUAL)
        emp = _tot_desp(EMP_ACUM)
        liq = _tot_desp(LIQ_ACUM)
        pag = _tot_desp(PAG_ACUM)

        saldo_emp = max(dot - emp, 0.0)
        saldo_liq = max(emp - liq, 0.0)
        saldo_pagar = max(liq - pag, 0.0)

        COR_EMP = "#A925EB"
        COR_LIQ = "#3030CE"
        COR_PAG = "#16A382"
        COR_SALDO = "#A9AFB6"

        k1, k2, k3, k4 = st.columns(4)
        k1.metric("Dotação Atualizada", _fmt_brl(dot))
        k2.metric("Empenhado (acum.)", _fmt_brl(emp))
        k3.metric("Liquidado (acum.)", _fmt_brl(liq))
        k4.metric("Pago (acum.)", _fmt_brl(pag))

        st.caption("Progresso por estágio")

        def bullet_dupla(titulo: str, atingido: float, meta: float, cor_atingido: str, saldo: float, help_txt: str):
            total = max(meta, atingido)
            total = total * 1.15 if total > 0 else 1

            ticks, ticktext, top_round, _ = auto_ticks(total, max_ticks=8)

            fig_b = go.Figure()

            fig_b.add_bar(
                x=[atingido], y=[titulo], orientation="h",
                marker=dict(color=cor_atingido),
                text=[_fmt_brl(atingido)], textposition="inside", insidetextanchor="middle",
                hovertemplate="Atingido: <b>%{text}</b><extra></extra>",
                name="Atingido"
            )

            if saldo > 1e-6:
                fig_b.add_bar(
                    x=[saldo], y=[titulo], orientation="h",
                    marker=dict(color=COR_SALDO),
                    text=[_fmt_brl(saldo)], textposition="outside", cliponaxis=False,
                    hovertemplate="Saldo: <b>%{text}</b><extra></extra>",
                    name="Saldo"
                )

            fig_b.update_layout(
                barmode="stack", template="simple_white",
                height=180, margin=dict(l=12, r=16, t=10, b=30),
                showlegend=False,
                xaxis=dict(
                    range=[0, top_round],
                    tickmode="array", tickvals=ticks, ticktext=ticktext,
                    zeroline=True, zerolinecolor="rgba(0,0,0,0.1)", showgrid=True
                ),
                yaxis=dict(title="", showgrid=False),
                plot_bgcolor="rgba(0,0,0,0)",
                hoverlabel=dict(bgcolor="white", font_size=12)
            )

            st.plotly_chart(fig_b, use_container_width=True)

        bullet_dupla("Dotação → Empenhado", emp, dot, COR_EMP, saldo_emp,
                     "Quanto do total autorizado (dotação) já foi empenhado.")
        bullet_dupla("Empenhado → Liquidado", liq, emp, COR_LIQ, saldo_liq,
                     "Quanto do empenhado já foi liquidado.")
        bullet_dupla("Liquidado → Pago", pag, liq, COR_PAG, saldo_pagar,
                     "Quanto do liquidado já foi pago.")

        # --- Top 10 Maiores Despesas — somente LIQUIDADO ---
        if not des.empty and "liq_acum" in des.columns:
            st.subheader("Maiores Despesas Liquidadas (acum.)")

            df = des.copy()
            df["conta"] = df["conta"].astype("string").str.strip()
            df["cod_conta"] = df["cod_conta"].astype("string").str.strip()
            df["liq_num"] = pd.to_numeric(df["liq_acum"], errors="coerce")

            m_val = df["liq_num"].notna() & (df["liq_num"] > 0)
            eh_agregador_codigo = df["cod_conta"].str.contains(
                r"^(Total|Subtotal|Saldo|Resultado)", case=False, na=False
            )

            ban_dupla = df["conta"].str.fullmatch(
                r"\s*DESPESAS\s+CORRENTES\s*|\s*DESPESAS\s+DE\s+CAPITAL\s*", case=False, na=False
            )

            import unicodedata
            def _norm(s: str) -> str:
                if not isinstance(s, str):
                    return ""
                s = unicodedata.normalize("NFD", s)
                s = "".join(ch for ch in s if unicodedata.category(ch) != "Mn")
                return s.upper().strip()

            conta_norm = df["conta"].astype(str).map(_norm)
            ban_extra = (
                conta_norm.str.contains(r"DESPESAS\s*\(EXCETO\s*INTRA[-\s]?ORCAMENTARIAS\)", regex=True, na=False) |
                conta_norm.str.contains(r"DESPESAS\s*\(INTRA[-\s]?ORCAMENTARIAS\)", regex=True, na=False) |
                conta_norm.str.contains(r"\bSUPERAVIT\b", regex=True, na=False)
            )

            tem_minuscula = df["conta"].str.contains(r"[a-záéíóúâêôãõç]", regex=True, na=False)

            folhas = df[m_val & ~tem_minuscula & ~eh_agregador_codigo & ~ban_dupla & ~ban_extra].copy()

            if folhas.empty:
                st.info("Não há contas detalhadas de despesa liquidadas para exibir.")
            else:
                agg = (folhas.groupby("conta", as_index=False, sort=False)["liq_num"].sum())
                topD = agg.nlargest(10, "liq_num").sort_values("liq_num", ascending=False)

                fig_topD = px.bar(
                    topD, x="liq_num", y="conta", orientation="h",
                    labels={"liq_num": "Liquidado (R$)", "conta": ""}
                )

                fig_topD.update_yaxes(
                    categoryorder="array",
                    categoryarray=topD["conta"].tolist(),
                    autorange="reversed"
                )

                ymax = float(topD["liq_num"].max())
                top_round, tickvals, ticktext = _ticks_dinamicos(ymax, margem=0.10)

                fig_topD.update_layout(
                    template="simple_white",
                    height=520,
                    margin=dict(l=12, r=16, t=10, b=10),
                )
                fig_topD.update_xaxes(
                    title="Liquidado (R$)",
                    range=[0, top_round],
                    tickmode="array", tickvals=tickvals, ticktext=ticktext,
                    zeroline=True, zerolinecolor="rgba(0,0,0,0.1)"
                )

                st.plotly_chart(fig_topD, use_container_width=True)

    # ============================================================
    # TAB RREO 2
    # ============================================================
    with tab_rreo02:
        st.subheader("Despesa por Função de Governo")

        df2 = preparar_rreo2(bases)
        if df2.empty:
            st.info("Sem dados do RREO Anexo 02 para esse contexto.")
            st.stop()

        funcoes = rreo2_so_funcoes(df2)
        if funcoes.empty:
            st.info("Não encontrei linhas de Função (apenas subfunções).")
            st.stop()

        _fmt = lambda v: "R$ " + f"{v:,.0f}".replace(",", ".")

        # KPIs – TOTAL (III)
        tot2 = totais_rreo2(bases)
        k1, k2, k3 = st.columns(3)
        k1.metric("Dotação Atualizada", _fmt(tot2["dot"]))
        k2.metric("Empenhado (acum.)", _fmt(tot2["emp"]))
        k3.metric("Liquidado (acum.)", _fmt(tot2["liq"]))

        st.divider()

        # Top 10 Funções — Liquidado
        st.write("#### 10 Maiores Funções — Liquidado (acum.)")
        top_fun = funcoes.nlargest(10, "liq_acum").sort_values("liq_acum")
        fig_f = px.bar(top_fun, x="liq_acum", y="conta", orientation="h",
                       labels={"liq_acum": "Liquidado (R$)", "conta": ""})
        ymax = float(top_fun["liq_acum"].max()) * 1.10
        tickvals, ticktext, top_round, _ = auto_ticks(ymax, max_ticks=8)
        fig_f.update_layout(template="simple_white", height=520, margin=dict(l=12, r=16, t=10, b=10))
        fig_f.update_xaxes(title="Liquidado (R$)", range=[0, top_round],
                           tickvals=tickvals, ticktext=ticktext,
                           zeroline=True, zerolinecolor="rgba(0,0,0,0.1)")
        st.plotly_chart(fig_f, use_container_width=True)

        st.divider()

        # Comparativo Empenhado × Liquidado — Top 10
        st.write("#### Empenhado × Liquidado — Top 10 Funções (por Liquidado)")
        cmp = funcoes.nlargest(10, "liq_acum").sort_values("liq_acum", ascending=False)
        COR_EMP = "#A925EB"
        COR_LIQ = "#3030CE"
        fig_cmp = go.Figure()
        fig_cmp.add_bar(name="Empenhado", x=cmp["conta"], y=cmp["emp_acum"], marker_color=COR_EMP)
        fig_cmp.add_bar(name="Liquidado", x=cmp["conta"], y=cmp["liq_acum"], marker_color=COR_LIQ)

        ymax = float(max(cmp["emp_acum"].max(), cmp["liq_acum"].max())) * 1.10
        tickvals, ticktext, top_round, _ = auto_ticks(ymax, max_ticks=8)

        fig_cmp.update_layout(
            barmode="group", template="simple_white", height=520,
            legend=dict(orientation="h", y=1.12, x=0.0, bgcolor="rgba(0,0,0,0)"),
            margin=dict(l=20, r=20, t=40, b=80),
            xaxis_title="", yaxis_title="R$",
            bargap=0.25, bargroupgap=0.12
        )
        fig_cmp.update_xaxes(tickangle=-30, automargin=True)
        fig_cmp.update_yaxes(range=[0, top_round], tickmode="array",
                             tickvals=tickvals, ticktext=ticktext,
                             zeroline=True, zerolinecolor="rgba(0,0,0,0.1)")
        st.plotly_chart(fig_cmp, use_container_width=True)

        st.divider()
        with st.expander("Ver tabela (Funções)"):
            st.dataframe(funcoes[["conta", "dot_atual", "emp_acum", "liq_acum"]], use_container_width=True)
